#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Pandas

Now that you had a quick introduction to Python, let's talk about Pandas. 

[*Pandas*](http://pandas.pydata.org/) is a column-oriented data analysis API. It's a great tool for handling and analyzing input data, and many ML frameworks support *Pandas* data structures as inputs.
Although a comprehensive introduction to the *Pandas* API would span many pages, the core concepts are fairly straightforward, and we'll present them below. For a more complete reference, the [*Pandas* docs site](http://pandas.pydata.org/pandas-docs/stable/index.html) contains extensive documentation and many tutorials.

This introduction is limited to the DataFrame and Series objects and the methods on those objects that support data analysis.

## Overview

### Learning Objectives

  * Gain an introduction to the `DataFrame` and `Series` data structures of the *Pandas* library
  * Access and manipulate data within a `DataFrame` and `Series`
  * Import CSV data into a *Pandas* `DataFrame`
  * Reindex a `DataFrame` to shuffle data

### Prerequisites

* Introduction to Python

### Estimated Duration

90 minutes

## Basic Concepts

The following line imports the *Pandas* API and prints the API version:

In [0]:
import pandas as pd
pd.__version__

The primary data structures in *Pandas* are implemented as two classes:

  * **`DataFrame`**, which you can imagine as a relational data table, with rows and named columns.
  * **`Series`**, which is a single column. A `DataFrame` contains one or more `Series` and a name for each `Series`.

The data frame is a commonly used abstraction for data manipulation. Similar implementations exist in [Spark](https://spark.apache.org/) and [R](https://www.r-project.org/about.html).

One way to create a `Series` is to construct a `Series` object. For example:

In [0]:
pd.Series(['San Francisco', 'San Jose', 'Sacramento'])

`DataFrame` objects can be created by passing a `dict` mapping `string` column names to their respective `Series`. If the `Series` don't match in length, missing values are filled with special [NA/NaN](http://pandas.pydata.org/pandas-docs/stable/missing_data.html) values. Example:

In [0]:
city_names = pd.Series(['San Francisco', 'San Jose', 'Sacramento'])
population = pd.Series([852469, 1015785, 485199])

pd.DataFrame({ 'City name': city_names, 'Population': population })

But most of the time, you load an entire file into a `DataFrame`. The following example loads a file with California housing data. Run the following cell to load the data and create feature definitions:

In [0]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")
california_housing_dataframe.describe()

The example above used `DataFrame.describe` to show interesting statistics about a `DataFrame`. Another useful function is `DataFrame.head`, which displays the first few records of a `DataFrame`:

In [0]:
california_housing_dataframe.head()

Another powerful feature of *Pandas* is graphing. For example, `DataFrame.hist` lets you quickly study the distribution of values in a column:

In [0]:
california_housing_dataframe.hist('housing_median_age')

## Accessing Data

You can access `DataFrame` data using familiar Python dict/list operations:

In [0]:
cities = pd.DataFrame({ 'City name': city_names, 'Population': population })
print(type(cities['City name']))
cities['City name']

In [0]:
print(type(cities['City name'][1]))
cities['City name'][1]

In [0]:
print(type(cities[0:2]))
cities[0:2]

In addition, *Pandas* provides an extremely rich API for advanced [indexing and selection](http://pandas.pydata.org/pandas-docs/stable/indexing.html) that is too extensive to be covered here.

## Manipulating Data

You may apply Python's basic arithmetic operations to `Series`. For example:

In [0]:
population / 1000.

[NumPy](http://www.numpy.org/) is a popular toolkit for scientific computing. *Pandas* `Series` can be used as arguments to most NumPy functions:

In [0]:
import numpy as np

np.log(population)

For more complex single-column transformations, you can use `Series.apply`. Like the Python [map function](https://docs.python.org/2/library/functions.html#map), 
`Series.apply` accepts as an argument a [lambda function](https://docs.python.org/2/tutorial/controlflow.html#lambda-expressions), which is applied to each value.

The example below creates a new `Series` that indicates whether `population` is over one million:

In [0]:
population.apply(lambda val: val > 1000000)


Modifying `DataFrames` is also straightforward. For example, the following code adds two `Series` to an existing `DataFrame`:

In [0]:
cities['Area square miles'] = pd.Series([46.87, 176.53, 97.92])
cities['Population density'] = cities['Population'] / cities['Area square miles']
cities

## Challenge

Modify the `cities` table by adding a new boolean column that is True if and only if *both* of the following are True:

  * The city is named after a saint.
  * The city has an area greater than 50 square miles.

**Note:** Boolean `Series` are combined using the bitwise, rather than the traditional boolean, operators. For example, when performing *logical and*, use `&` instead of `and`.

**Hint:** "San" in Spanish means "saint."

In [0]:
# Your code here

### Solution

Click below for a solution.

In [0]:
cities['Is wide and has saint name'] = (cities['Area square miles'] > 50) & cities['City name'].apply(lambda name: name.startswith('San'))
cities

## Indexes
Both `Series` and `DataFrame` objects also define an `index` property that assigns an identifier value to each `Series` item or `DataFrame` row. 

By default, at construction, *Pandas* assigns index values that reflect the ordering of the source data. Once created, the index values are stable; that is, they do not change when data is reordered.

In [0]:
city_names.index

In [0]:
cities.index

Call `DataFrame.reindex` to manually reorder the rows. For example, the following has the same effect as sorting by city name:

In [0]:
cities.reindex([2, 0, 1])

Reindexing is a great way to shuffle (randomize) a `DataFrame`. In the example below, we take the index, which is array-like, and pass it to NumPy's `random.permutation` function, which shuffles its values in place. Calling `reindex` with this shuffled array causes the `DataFrame` rows to be shuffled in the same way.
Try running the following cell multiple times!

In [0]:
cities.reindex(np.random.permutation(cities.index))

For more information, see the [Index documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#index-objects).

## Challenge

The `reindex` method allows index values that are not in the original `DataFrame`'s index values. Try it and see what happens if you use such values! Why do you think this is allowed? Does reindexing effect the underlying data structure or does it make a copy?

In [0]:
# Your code here

### Solution

Click below for the solution.

If your `reindex` input array includes values not in the original `DataFrame` index values, `reindex` will add new rows for these "missing" indices and populate all corresponding columns with `NaN` values:

In [0]:
cities.reindex([0, 4, 5, 2])

This behavior is desirable because indexes are often strings pulled from the actual data (see the [*Pandas* reindex
documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reindex.html) for an example
in which the index values are browser names).

In this case, allowing "missing" indices makes it easy to reindex using an external list, as you don't have to worry about
sanitizing the input.

# Exercises

## Exercise 1

1.   Visit [data.gov's CSV datasets](https://catalog.data.gov/dataset?res_format=CSV) and find "Demographic Statistics By Zip Code: City of New York — Demographic statistics broken down by zip code".
2.   Find the CSV download URL for the dataset and use that URL to download the CSV into a DataFrame.
3.   Use *Pandas* to "describe" the data.

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
import pandas as pd

ny_stats = pd.read_csv('https://data.cityofnewyork.us/api/views/kku6-nxdu/rows.csv?accessType=DOWNLOAD')
ny_stats.describe()

**Validation**

In [0]:
# TODO

## Exercise 2

Use the *Pandas* `head` method to look at the first 20 rows of data.

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
ny_stats.head(20)

**Validation**

In [0]:
# TODO

## Exercise 3

Use the *Pandas* `tail` method to look at the last 20 rows of data.

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
ny_stats.tail(20)

**Validation**

In [0]:
# TODO

## Exercise 4

There are over 40 columns of data in this data frame. It is difficult to read them horizontally. Write some *Pandas* code that prints the name of each column (*Pandas* series). Print one name per line.

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
list(ny_stats)

**Validation**

In [0]:
# TODO

## Exercise 5

1.  Zip codes should be unique in this dataset. Verify that is true.
1.  Reindex the data frame using zip code as the index.
1.  Use *Pandas* to write code to find all of the zip codes where there is no data (all zeros) in all columns except the zip code.
1.  Remove these rows of data from the DataFrame.

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
# 1. Zip codes should be unique in this dataset. Verify that is true.
assert(ny_stats['JURISDICTION NAME'].is_unique)

# 2. Reindex the data frame using zip code as the index.
ny_stats.set_index('JURISDICTION NAME', inplace=True)
ny_stats.head()

# 3. Use Pandas to write code to find all of the zip codes where there is no 
# data (all zeros) in all columns except the zip code.
ny_stats[ny_stats.sum(axis=1) == 0]

# 4. Remove these rows of data from the DataFrame.
ny_stats = ny_stats[ny_stats.sum(axis=1) != 0]
ny_stats.describe()

**Validation**

In [0]:
# TODO

## Exercise 6

Examine the dataset and try to do some research to find out how the data was collected. Try to identify problems with the dataset related to data quality, data completeness, and bias.

### Student Solution

*Your answer goes here (English, not Python!)*

### Answer Key

**Solution**

TODO

**Validation**

In [0]:
# N/A